In [ ]:
IMAGE_DIR = "images"
FOLDERS=["happy","sad","relaxed"]
DEVICE = "cuda"


In [ ]:
import pandas as pd
import os

In [ ]:
img_files =[]
labels=[]
label_code =[]
for folder in FOLDERS:
    fname=os.path.join(IMAGE_DIR,folder)
    for im in os.listdir(fname):
        impath= os.path.join(fname,im)
        img_files.append(impath)
        labels.append(folder)
        label_code.append(FOLDERS.index(folder))
        


In [ ]:
dataset=pd.DataFrame(dict(filename=img_files,label=labels,code=label_code))
dataset

,filename,label,code
0,images\happy\10005733405_81d43d7c81_b.jpg,happy,0
1,images\happy\10009139114_6909bb23a4_b.jpg,happy,0
2,images\happy\100198545_1e8a8f2b09_b.jpg,happy,0
3,images\happy\10027367755_c3ee04c899_b.jpg,happy,0
4,images\happy\10041603564_4d95f98631_b.jpg,happy,0
...,...,...,...
13660,images\relaxed\9885101675_8aaa6fd7f0_b.jpg,relaxed,2
13661,images\relaxed\990109189_6bfd95057b_b.jpg,relaxed,2
13662,images\relaxed\99453002_bdd708cbbf_b.jpg,relaxed,2
13663,images\relaxed\9960487016_33df209c9b_b.jpg,relaxed,2


In [ ]:
import torch


In [ ]:
torch.manual_seed(0)

In [ ]:
torch.cuda.is_available()


True

In [ ]:
# torch.bckends.cuda.is_available()

In [ ]:
device=torch.device(DEVICE)

In [ ]:
from torchvision.io import read_image
from torchvision import transforms as T
from torch.utils.data import Dataset
import math

In [ ]:
class DogDataset(Dataset):
    def __init__(self,dataset):
        self.dataset=dataset
        self.augments=[T.RandomHorizontalFlip(1),T.RandomRotation(90),T.AutoAugment()]
        self.normalize=T.Compose([
            T.ConvertImageDtype(torch.float),
            T.Normalize((.485,.456,0.406),(.229,.224,.225))

        ])
    def __len__(self):
        return self.dataset.shape[0]*(len(self.augments)+1)
    def classes(self):
        return self.dataset["code"].unique()
    def __getitem__(self,idx):
        augment=math.floor(idx/self.dataset.shape[0])
        idx=idx%self.dataset.shape[0]
        row=self.dataset.iloc[idx,:]
        img_path=row["filename"]
        image=read_image(img_path)

        label=row["code"]
        if augment>0:
            image=self.augments[augment-1].forward(image)
        image = self.normalize(image)
        return image, int(label),img_path


In [ ]:
data =DogDataset(dataset)

In [ ]:
train_size=int(0.8 * len(data))
test_size =len(data) - train_size
train_data,test_data = torch.utils.data.random_split(data,[train_size,test_size], generator =torch.Generator().manual_seed(2))

In [ ]:
train_data

In [ ]:
from torch.utils.data import DataLoader
BATCH_SIZE = 64
EPOCHS = 50
train = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)
test = DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=True)

In [ ]:
from torch import nn
class NeuralNetwork(nn.Module):
    def __init__(self,classes):
        super(NeuralNetwork,self).__init__()
        self.bn=nn.BatchNorm2d(64)

        self.cnn=nn.Sequential(
            nn.Conv2d(3,64,4,stride=2),
            self.bn,
            nn.ReLU(True),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64,64,2),
            self.bn,
            nn.Conv2d(64,64,2),
            self.bn,
            nn.ReLU(True),
            nn.MaxPool2d(2,2)
        )
        self.dense=nn.Sequential(
            nn.Linear(64*46*46,64),
            nn.Linear(64,len(classes))
        )
    def forward(self,x):
        x=self.cnn(x)
        x=torch.flatten(x,1)
        x=self.dense(x)
        return x
    



In [ ]:
labels=data.classes()


In [ ]:
labels

array([0, 1, 2], dtype=int64)

In [ ]:
model =NeuralNetwork(labels).to(device)

In [ ]:
model

NeuralNetwork(
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=135424, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=3, bias=True)
  )
)

In [ ]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.001)

In [ ]:
size=len(train.dataset)

In [ ]:
for epoch in range(EPOCHS):
    for batch,(images,labels,img_paths) in enumerate(train):
        optimizer.zero_grad()
        images=images.to(device)
        pred=model(images.float())
        labels=labels.to(device)
        loss=loss_fn(pred,labels)
        loss.backward()
        optimizer.step()
    loss=loss.item()
    print(f"loss: {loss:>7f} [{epoch}]")


loss: 0.927391 [0]
loss: 0.918224 [1]
loss: 0.771304 [2]
loss: 0.725238 [3]
loss: 0.926150 [4]
loss: 0.758242 [5]
loss: 0.822362 [6]
loss: 0.808502 [7]
loss: 0.614932 [8]
loss: 0.572946 [9]
loss: 0.855120 [10]
loss: 0.815718 [11]
loss: 0.589736 [12]
loss: 0.671926 [13]
loss: 0.564206 [14]
loss: 0.730876 [15]
loss: 0.610559 [16]
loss: 0.715572 [17]
loss: 0.676569 [18]
loss: 0.623424 [19]
loss: 1.040449 [20]
loss: 0.712234 [21]
loss: 1.106371 [22]
loss: 0.633634 [23]
loss: 0.751319 [24]
loss: 0.601703 [25]
loss: 0.516777 [26]
loss: 0.550158 [27]
loss: 0.691137 [28]
loss: 0.592026 [29]
loss: 0.971106 [30]
loss: 0.419768 [31]
loss: 0.672401 [32]
loss: 0.792418 [33]
loss: 0.403452 [34]
loss: 0.704606 [35]
loss: 0.832409 [36]
loss: 0.710119 [37]
loss: 0.531632 [38]
loss: 0.638917 [39]
loss: 0.539979 [40]
loss: 0.768462 [41]
loss: 0.463059 [42]
loss: 0.681114 [43]
loss: 0.589770 [44]
loss: 0.798372 [45]
loss: 0.491516 [46]
loss: 0.446718 [47]
loss: 0.662856 [48]
loss: 0.655537 [49]


In [ ]:
torch.save(model.state_dict(),"Dil_gabara.ptl")

In [ ]:
torch.save(model.state_dict(),"model1.pth")

In [ ]:
torch.save(model.state_dict(),"model2.pt")

In [ ]:
torch.save(model,"cnmod.pth")

In [ ]:
# modx=NeuralNetwork(labels)
# modx.load(torch.load("S:\DL projects\model1.pth"))
# torch.save(model.state_dict(), PATH)
# Load:

# model = TheModelClass(*args, **kwargs)
# model.load_state_dict(torch.load(PATH))
# model.eval()
modx=torch.load("S:\DL projects\cnmod.pth")

In [ ]:
all_preds=list()
all_labels=list()
all_paths=list()
with torch.no_grad():
    for batch, (images, labels,img_paths) in enumerate(test):
        images=images.to(device)
        outputs=modx(images.float())
        _, preds = torch.max(outputs.data, 1)
        all_labels.append(labels)
        all_preds.append(preds)
        all_paths.append(img_paths)
# for epoch in range(EPOCHS):
#     for batch,(images,labels,img_paths) in enumerate(train):
#         optimizer.zero_grad()
#         images=images.to(device)
#         pred=model(images.float())
#         labels=labels.to(device)
#         loss=loss_fn(pred,labels)
#         loss.backward()
#         optimizer.step()
#     loss=loss.item()
#     print(f"loss: {loss:>7f} [{epoch}]")


In [ ]:
import numpy as np
preds=np.concatenate([p.cpu().numpy() for p in all_preds])
labels=np.concatenate([p.cpu().numpy() for p in all_labels])
paths=np.concatenate([p for p in all_paths])


In [ ]:
((preds==labels).sum()) /len(labels)

0.33699231613611413

In [ ]:
predictions=pd.DataFrame(dict(pred=preds,label=labels,path=paths))
predictions["correct"]=(predictions["pred"]==predictions["label"])

In [ ]:
predictions["prediction"]=predictions["pred"].apply(lambda x:FOLDERS[int(x)])
predictions["actual"]=predictions["label"].apply(lambda x:FOLDERS[int(x)])


In [ ]:
predictions.groupby("prediction").apply(lambda x:x["correct"].sum()/x.shape[0])

prediction
happy      0.357688
relaxed    0.317840
sad        0.333617
dtype: float64

In [ ]:
disp=predictions.iloc[5255:5355,:].copy()


In [ ]:
disp

,pred,label,path,correct,prediction,actual
5255,0,2,images\relaxed\8618354972_289ace1179_b.jpg,False,happy,relaxed
5256,1,0,images\happy\9876232576_6de4ca82de_b.jpg,False,sad,happy
5257,2,1,images\sad\4846713446_711ff90721_b.jpg,False,relaxed,sad
5258,0,2,images\relaxed\6316538173_4729caa553_b.jpg,False,happy,relaxed
5259,0,0,images\happy\27122007695_24fb84bc8a_b.jpg,True,happy,happy
...,...,...,...,...,...,...
5350,0,0,images\happy\7913542232_a570d4a16b_b.jpg,True,happy,happy
5351,0,2,images\relaxed\26779472488_e3090a6da7_b.jpg,False,happy,relaxed
5352,2,2,images\relaxed\8421473767_91112b49e3_b.jpg,True,relaxed,relaxed
5353,2,1,images\sad\6095669840_b2cf1e9735_b.jpg,False,relaxed,sad


In [ ]:
def image_formatter(path):
    return f'<img src="{path}">'

In [ ]:
disp.style.format({'path':image_formatter})

,pred,label,path,correct,prediction,actual
5255,0,2,,False,happy,relaxed
5256,1,0,,False,sad,happy
5257,2,1,,False,relaxed,sad
5258,0,2,,False,happy,relaxed
5259,0,0,,True,happy,happy
5260,0,1,,False,happy,sad
5261,2,0,,False,relaxed,happy
5262,0,1,,False,happy,sad
5263,1,1,,True,sad,sad
5264,1,1,,True,sad,sad
